#Project Kojak

In [2]:
%load_ext Cython

In [3]:
%%cython

cimport numpy as np
import pandas as pd
import numpy as np
import math

In [8]:
def time_exp(tj, ti):
    return math.exp(-(tj - ti))

In [9]:
def space_exp(xj, xi, yj, yi): 
    return math.exp(- ((xj-xi)**2 + (yj - yi)**2) / 2)

In [4]:
%%cython -a

def background_kernel(x,y):
    '''
    space_matrix is a list of all space matrices of each category
    '''
    total_sum = 0
    for i in range(0,6):
        total_sum += alpha[i] * np.exp(space_matrix[i]**(-1/(2*eta_squared)))
    
    return total_sum / (T * 2 * np.pi * eta_squared)


Error compiling Cython file:
------------------------------------------------------------
...
    '''
    space_matrix is a list of all space matrices of each category
    '''
    total_sum = 0
    for i in range(0,6):
        total_sum += alpha[i] * np.exp(space_matrix[i]**(-1/(2*eta_squared)))
                         ^
------------------------------------------------------------

/Users/catherinedupuis/.ipython/cython/_cython_magic_f91f4f6a883c45515b75883e1e72413d.pyx:8:26: undeclared name not builtin: alpha

Error compiling Cython file:
------------------------------------------------------------
...
    '''
    space_matrix is a list of all space matrices of each category
    '''
    total_sum = 0
    for i in range(0,6):
        total_sum += alpha[i] * np.exp(space_matrix[i]**(-1/(2*eta_squared)))
                                 ^
------------------------------------------------------------

/Users/catherinedupuis/.ipython/cython/_cython_magic_f91f4f6a883c45515b75883e1e72413d.p

In [11]:
def triggering_kernel(x,y,t):
    '''
    space_matrix is a list of all space matrices of each category
    time_matrix is a list of all time matrices of each category
    '''
    total_sum = 0
    for i in range(0,6):
        total_sum += theta[i] * np.exp(time_matrix[i]**(- omega)) * np.exp(space_matrix[i]**(-1/(2 * sigma_squared)))
        
    return (total_sum * omega) / (2 * np.pi * sigma_squared)

In [12]:
def intensity(x, y, t):
    return background_kernel(x,y) + triggering_kernel(x,y,t)

In [13]:
def trigger_probability(i,j,category):
    #category should be an array of the category with time, x and y values
    part1 = (theta[i] * omega * time_matrix[i,j]**omega * space_matrix**sigma_squared) / (2*math.Pi*sigma_squared)
    total = part1 / intensity(category[1][j], category[2][j], category[0][j])
    return total

In [14]:
def background_probability(i, j, category):
    #category should be an array of the category with time, x and y values
    part1 = (alpha[i] * space_matrix[i,j]**eta_squared) / (2 * np.pi * eta_squared)
    total = part1 / (T * intensity(category[1][j],category[2][j],category[0][j]))
    return total

In [15]:
def calculate_omega():
    part1 = trigger_matrix.sum()
    part2 = np.multiply(trigger_matrix,time_matrix).sum()
    part3 = 0
    return part1 / (part2 + part3)

In [16]:
def calculat_sigma_squared():
    part1 = 0
    part2 = 0
    return part1 / part2

In [17]:
def calculate_eta_squared():
    part1 = 0
    part2 = 0
    return part1 / part2

In [18]:
def calculate_theta(i):
    part1 = 0
    part2 = 0
    return part1 / part2

In [19]:
def calculate_alpha(i):
    part1 = 0
    return part1 / len(category[i]) #divide by number of events of this category

In [20]:
cpd_df = pd.read_csv("CPD-model-data-2007-2012")

In [21]:
cpd_df['Date'] = pd.to_datetime(cpd_df['Date'])

In [22]:
cpd_df.head()

,Unnamed: 0,Date,Block,Primary Type,Description,Location Description,Arrest,Domestic,District,Ward,Community Area,Latitude,Longitude,Location
0,886855,2013-01-01 00:00:00,076XX S UNION AVE,CRIM SEXUAL ASSAULT,PREDATORY,APARTMENT,True,False,6,17,71,41.755367,-87.641721,"(41.755366504, -87.641720553)"
1,886861,2013-01-01 00:00:00,078XX S ESSEX AVE,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,True,4,7,43,41.753049,-87.563831,"(41.753049024, -87.563831077)"
2,886891,2013-01-01 00:00:00,012XX S SPRINGFIELD AVE,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,False,10,24,29,41.865523,-87.722716,"(41.865522673, -87.722715696)"
3,886914,2012-12-31 23:45:00,026XX N CALIFORNIA AVE,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,14,35,22,41.929360,-87.697544,"(41.929359781, -87.697544118)"
4,886915,2012-12-31 23:45:00,070XX S ELIZABETH ST,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,7,17,67,41.765882,-87.655350,"(41.76588157, -87.655349847)"


In [23]:
np.sort(cpd_df['Primary Type'].unique())

array(['ASSAULT', 'BATTERY', 'CRIM SEXUAL ASSAULT', 'HOMICIDE', 'ROBBERY',
       'WEAPONS VIOLATION'], dtype=object)

In [24]:
len(cpd_df[cpd_df['Primary Type'] == 'ASSAULT'])

109661

In [12]:
print("Number of Homicides: %i" % len(cpd_df[cpd_df['Primary Type'] == 'HOMICIDE']))
print("Number of Robberies: %i" % len(cpd_df[cpd_df['Primary Type'] == 'ROBBERY']))
print("Number of Assaults: %i" % len(cpd_df[cpd_df['Primary Type'] == 'ASSAULT']))
print("Number of Weapons violation: %i" % len(cpd_df[cpd_df['Primary Type'] == 'WEAPONS VIOLATION']))
print("Number of Batteries: %i" % len(cpd_df[cpd_df['Primary Type'] == 'BATTERY']))
print("Number of Sexual Assaults: %i" % len(cpd_df[cpd_df['Primary Type'] == 'CRIM SEXUAL ASSAULT']))

Number of Homicides: 2800
Number of Robberies: 74123
Number of Assaults: 109661
Number of Weapons violation: 19369
Number of Batteries: 328004
Number of Sexual Assaults: 6802


In [14]:
print(np.sort(cpd_df[cpd_df['Primary Type'] == 'ROBBERY']['Description'].unique()))
print(np.sort(cpd_df[cpd_df['Primary Type'] == 'ASSAULT']['Description'].unique()))
print(np.sort(cpd_df[cpd_df['Primary Type'] == 'WEAPONS VIOLATION']['Description'].unique()))
print(np.sort(cpd_df[cpd_df['Primary Type'] == 'BATTERY']['Description'].unique()))
print(np.sort(cpd_df[cpd_df['Primary Type'] == 'CRIM SEXUAL ASSAULT']['Description'].unique()))

['AGGRAVATED' 'AGGRAVATED VEHICULAR HIJACKING' 'ARMED: HANDGUN'
 'ARMED: OTHER DANGEROUS WEAPON' 'ARMED: OTHER FIREARM'
 'ARMED:KNIFE/CUTTING INSTRUMENT' 'ATTEMPT: AGGRAVATED'
 'ATTEMPT: ARMED-HANDGUN' 'ATTEMPT: ARMED-KNIFE/CUT INSTR'
 'ATTEMPT: ARMED-OTHER DANG WEAP' 'ATTEMPT: ARMED-OTHER FIREARM'
 'ATTEMPT: STRONGARM-NO WEAPON' 'STRONGARM - NO WEAPON'
 'VEHICULAR HIJACKING']
['AGG PO HANDS NO/MIN INJURY' 'AGG PRO.EMP: HANDGUN'
 'AGG PRO.EMP: OTHER DANG WEAPON' 'AGG PRO.EMP: OTHER FIREARM'
 'AGG PRO.EMP:KNIFE/CUTTING INST' 'AGGRAVATED PO: HANDGUN'
 'AGGRAVATED PO: OTHER DANG WEAP' 'AGGRAVATED PO: OTHER FIREARM'
 'AGGRAVATED PO:KNIFE/CUT INSTR' 'AGGRAVATED: HANDGUN'
 'AGGRAVATED: OTHER DANG WEAPON' 'AGGRAVATED: OTHER FIREARM'
 'AGGRAVATED:KNIFE/CUTTING INSTR' 'PRO EMP HANDS NO/MIN INJURY' 'SIMPLE']
['DEFACE IDENT MARKS OF FIREARM' 'POSS FIREARM/AMMO:NO FOID CARD'
 'RECKLESS FIREARM DISCHARGE' 'REGISTER OF SALES BY DEALER'
 'UNLAWFUL POSS AMMUNITION' 'UNLAWFUL POSS OF HANDGUN'
 'UNLAWFU

In [15]:
print(len(cpd_df[cpd_df['Primary Type'] == 'ROBBERY'][cpd_df['Description'].str.contains("HANDGUN")]))
print(len(cpd_df[cpd_df['Primary Type'] == 'ASSAULT'][cpd_df['Description'].str.contains("HANDGUN")]))
print(len(cpd_df[cpd_df['Primary Type'] == 'WEAPONS VIOLATION'][cpd_df['Description'].str.contains("HANDGUN")]))
print(len(cpd_df[cpd_df['Primary Type'] == 'BATTERY'][cpd_df['Description'].str.contains("HANDGUN")]))
print(len(cpd_df[cpd_df['Primary Type'] == 'CRIM SEXUAL ASSAULT'][cpd_df['Description'].str.contains("HANDGUN")]))

28957
11718
13809
9069
500


/Applications/anaconda/lib/python3.4/site-packages/pandas/core/frame.py:1825: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


In [25]:
categories = []
categories.append(cpd_df[cpd_df['Primary Type'] == 'HOMICIDE'])
categories.append(cpd_df[cpd_df['Primary Type'] == 'ROBBERY'][cpd_df['Description'].str.contains("HANDGUN")])
categories.append(cpd_df[cpd_df['Primary Type'] == 'ASSAULT'][cpd_df['Description'].str.contains("HANDGUN")])
categories.append(cpd_df[cpd_df['Primary Type'] == 'WEAPONS VIOLATION'][cpd_df['Description'].str.contains("HANDGUN")])
categories.append(cpd_df[cpd_df['Primary Type'] == 'BATTERY'][cpd_df['Description'].str.contains("HANDGUN")])
categories.append(cpd_df[cpd_df['Primary Type'] == 'CRIM SEXUAL ASSAULT'][cpd_df['Description'].str.contains("HANDGUN")])

/Applications/anaconda/lib/python3.4/site-packages/pandas/core/frame.py:1825: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


In [26]:
for i in range(0,len(categories)):
    categories[i] = categories[i][['Primary Type','Date','Latitude','Longitude']].sort('Date').reset_index(drop=True)

In [27]:
for i in range(0,len(categories)):
    categories[i] = categories[i].as_matrix(columns=categories[i].columns[1:])

In [161]:
td = categories[0][7][0] - categories[0][0][0]
td

Timedelta('7 days 21:15:00')

In [10]:
cat = categories[0]
cat[0][1]

41.766615573

In [77]:
num_homicides = len(categories[0])
total_num_crimes = len(categories[0]) + len(categories[1]) + len(categories[2]) + len(categories[3]) + len(categories[4]) + len(categories[5])
print(num_homicides)
print(total_num_crimes)

2800
66853


In [115]:
'''
Should be arranged in order: Homicides, Robberies, Assaults, Weapons violation, Battery, Crim Sexual Assault
'''
category_times = []
for i in range(0,6):
    category_times.append([cat[0].timestamp() for cat in categories[i]])

In [118]:
time_matrix = []
for i in range(0,6):
    temp_time_matrix = np.array([category_times[0],]*len(category_times[i])) - np.array([category_times[i],]*len(category_times[0])).transpose()
    temp_time_matrix[temp_time_matrix < 0] = 0
    time_matrix.append(temp_time_matrix)

In [126]:
time_matrix[5] # time_matrix for sexual assaults

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          1.56343200e+08,   1.56368760e+08,   1.56428340e+08],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          1.55895000e+08,   1.55920560e+08,   1.55980140e+08],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          1.55430600e+08,   1.55456160e+08,   1.55515740e+08],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          7.67400000e+05,   7.92960000e+05,   8.52540000e+05],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          1.95000000e+05,   2.20560000e+05,   2.80140000e+05],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          9.33000000e+04,   1.18860000e+05,   1.78440000e+05]])

In [28]:
category_X = []
category_Y = []
for i in range(0,6):
    category_X.append([cat[1] for cat in categories[i]])
    category_Y.append([cat[2] for cat in categories[i]])

In [ ]:
space_matrix = []
for i in range(0,6):
    temp_X_matrix = np.array([category_X[0],]*len(category_X[i])) - np.array([category_X[i],]*len(category_X[0])).transpose()
    temp_Y_matrix = np.array([category_Y[0],]*len(category_Y[i])) - np.array([category_Y[i],]*len(category_Y[0])).transpose()
    temp_time_matrix = temp_X_matrix**2 + temp_Y_matrix**2
    space_matrix.append(temp_time_matrix)

In [ ]:
temp_X_matrix = np.array([category_X[0],]*len(category_X[1])) - np.array([category_X[1],]*len(category_X[0])).transpose()
temp_Y_matrix = np.array([category_Y[0],]*len(category_Y[1])) - np.array([category_Y[1],]*len(category_Y[0])).transpose()
temp_time_matrix = temp_X_matrix**2 + temp_Y_matrix**2

________

In [5]:
from osgeo import ogr

ImportError: No module named 'osgeo'